In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import gc

In [2]:
ls -lh '/kaggle/input/playground-series-s4e9'

total 46M
-rw-r--r-- 1 nobody nogroup 2.1M Sep  1 00:08 sample_submission.csv
-rw-r--r-- 1 nobody nogroup  18M Sep  1 00:08 test.csv
-rw-r--r-- 1 nobody nogroup  27M Sep  1 00:08 train.csv


In [3]:
train_data= pd.read_csv("/kaggle/input/playground-series-s4e9/train.csv", index_col="id")
test_data = pd.read_csv("/kaggle/input/playground-series-s4e9/test.csv", index_col="id")
submission = pd.read_csv("/kaggle/input/playground-series-s4e9/sample_submission.csv")

In [4]:
from IPython.display import display_html
df1_styler = train_data.head().style.set_table_attributes("style='display:inline'").set_caption('Head Train Data').background_gradient(cmap='plasma')
df2_styler = test_data.head().style.set_table_attributes("style='display:inline'").set_caption('Head Test Data').background_gradient(cmap='plasma')
display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw = True)

Head Train Data 
 
 
   
 brand 
 model 
 model_year 
 milage 
 fuel_type 
 engine 
 transmission 
 ext_col 
 int_col 
 accident 
 clean_title 
 price 
 
 
 id 
   
   
   
   
   
   
   
   
   
   
   
   
 
 
 
 
 0 
 MINI 
 Cooper S Base 
 2007 
 213000 
 Gasoline 
 172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel 
 A/T 
 Yellow 
 Gray 
 None reported 
 Yes 
 4200 
 
 
 1 
 Lincoln 
 LS V8 
 2002 
 143250 
 Gasoline 
 252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel 
 A/T 
 Silver 
 Beige 
 At least 1 accident or damage reported 
 Yes 
 4999 
 
 
 2 
 Chevrolet 
 Silverado 2500 LT 
 2002 
 136731 
 E85 Flex Fuel 
 320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capability 
 A/T 
 Blue 
 Gray 
 None reported 
 Yes 
 13900 
 
 
 3 
 Genesis 
 G90 5.0 Ultimate 
 2017 
 19500 
 Gasoline 
 420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel 
 Transmission w/Dual Shift Mode 
 Black 
 Black 
 None reported 
 Yes 
 45000 
 
 
 4 
 Mercedes-Benz 
 Metris Base 
 2021 
 7388 
 Gasoline 
 208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel 
 7-Speed A/T 
 Black 
 Beige 
 None reported 
 Yes 
 97500 
 
 
 

 
 Head Test Data 
 
 
   
 brand 
 model 
 model_year 
 milage 
 fuel_type 
 engine 
 transmission 
 ext_col 
 int_col 
 accident 
 clean_title 
 
 
 id 
   
   
   
   
   
   
   
   
   
   
   
 
 
 
 
 188533 
 Land 
 Rover LR2 Base 
 2015 
 98000 
 Gasoline 
 240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel 
 6-Speed A/T 
 White 
 Beige 
 None reported 
 Yes 
 
 
 188534 
 Land 
 Rover Defender SE 
 2020 
 9142 
 Hybrid 
 395.0HP 3.0L Straight 6 Cylinder Engine Gasoline/Mild Electric Hybrid 
 8-Speed A/T 
 Silver 
 Black 
 None reported 
 Yes 
 
 
 188535 
 Ford 
 Expedition Limited 
 2022 
 28121 
 Gasoline 
 3.5L V6 24V PDI DOHC Twin Turbo 
 10-Speed Automatic 
 White 
 Ebony 
 None reported 
 nan 
 
 
 188536 
 Audi 
 A6 2.0T Sport 
 2016 
 61258 
 Gasoline 
 2.0 Liter TFSI 
 Automatic 
 Silician Yellow 
 Black 
 None reported 
 nan 
 
 
 188537 
 Audi 
 A6 2.0T Premium Plus 
 2018 
 59000 
 Gasoline 
 252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel 
 A/T 
 Gray 
 Black 
 None reported 
 Yes

In [5]:
def summary(df):
    print(f'data shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['data type'])
    summ['#missing'] = df.isnull().sum().values
    summ['#missing'] = df.isnull().sum().values / len(df) * 100
    summ['#unique'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
    summ['first value'] = df.loc[0].values
    summ['second value'] = df.loc[1].values
    summ['third value'] = df.loc[2].values
    
    return summ

In [6]:
summary(train_data).style.background_gradient(cmap='plasma')

data shape: (188533, 12)


,data type,#missing,#unique,min,max,first value,second value,third value
brand,object,0.000000,57,nan,nan,MINI,Lincoln,Chevrolet
model,object,0.000000,1897,nan,nan,Cooper S Base,LS V8,Silverado 2500 LT
model_year,int64,0.000000,34,1974.000000,2024.000000,2007,2002,2002
milage,int64,0.000000,6651,100.000000,405000.000000,213000,143250,136731
fuel_type,object,2.696080,7,nan,nan,Gasoline,Gasoline,E85 Flex Fuel
engine,object,0.000000,1117,nan,nan,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capability
transmission,object,0.000000,52,nan,nan,A/T,A/T,A/T
ext_col,object,0.000000,319,nan,nan,Yellow,Silver,Blue
int_col,object,0.000000,156,nan,nan,Gray,Beige,Gray
accident,object,1.300568,2,nan,nan,None reported,At least 1 accident or damage reported,None reported
